### Overview

Datathon 2024 başlıyor! Kurallar, tanım, değerlendirme ve kod kısımlarını detaylı olarak incelemeyi unutmayın!

**Datathon 2024 Ana Görev**

Verilerde Girişimcilik Vakfı'nın 2014 yılından itibaren aldığı başvuruları ve Değerlendirme Puanı isimli kolonu gözlemleyebilirsiniz (train.csv). Bu dosyanın içerisinde aynı zamanda başvuran kişilerin anonim olarak paylaşılmış üniversite, aile detayları, ikamet detayları vb. bir çok bilgiyi gözlemleyebilirsiniz.

2023 yılında başvuran 11.049 kişinin ise Değerlendirme Puanı kolonu hariç yeniden tüm verilerini bulabilirsiniz (test_x.csv). Görevimiz bu 11.049 kişinin Değerlendirme Puanı isimli kolonunu tahmin etmektir.

Örnek submission dosyasında olduğu gibi id, Degerlendirme Puani isimli 2 kolondan oluşan submission dosyanızda 11.049 kişinin skorları yer almalıdır (sample_submission.csv). 11.049 kişinin id'leri test_x.csv dosyasında olduğu sırayla verilmelidir.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train=pd.read_csv("train.csv")
train.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4


In [3]:
test=pd.read_csv("test.csv")
test.head()

,Basvuru Yili,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2023,Kadın,21.06.2002 00:00,Amasya,Amasya,Munzur Üniversitesi,Devlet,NaN,Hayır,Sosyal Hizmet,...,Takım Oyuncusu,Evet,Kızılay da gönüllü dağıtım,NaN,Hayır,-,Evet,NaN,NaN,0
1,2023,Kadın,11.05.2004 00:00,Konya,Konya,Hacettepe Üniversitesi,Devlet,NaN,Hayır,Uluslararası İlişkiler,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,1
2,2023,Erkek,24.04.2002 00:00,İstanbul,İstanbul,Kapadokya Üniversitesi,Özel,NaN,Evet,Diğer,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,2
3,2023,Erkek,11.03.2003 00:00,Mardin,Mardin,Mardin Artuklu Üniversitesi,Devlet,NaN,Hayır,İslami İlimler,...,-,Evet,Lise gezilerde liderlik yapıyordum,NaN,Hayır,-,Hayır,NaN,NaN,3
4,2023,Erkek,26.01.2002 00:00,Samsun,İstanbul,Boğaziçi Üniversitesi,Devlet,NaN,Hayır,İşletme,...,-,Evet,"Toplum Gönüllüleri Vakfı'nda ""Yol Arkadaşım"" P...",NaN,Evet,"12 aydır, yolculuğuna bir yıl önce başlamış ol...",Evet,NaN,NaN,4


In [4]:
train.columns

Index(['Basvuru Yili', 'Degerlendirme Puani', 'Cinsiyet', 'Dogum Tarihi',
       'Dogum Yeri', 'Ikametgah Sehri', 'Universite Adi', 'Universite Turu',
       'Burslu ise Burs Yuzdesi', 'Burs Aliyor mu?', 'Bölüm',
       'Universite Kacinci Sinif', 'Universite Not Ortalamasi',
       'Daha Once Baska Bir Universiteden Mezun Olmus', 'Lise Adi',
       'Lise Adi Diger', 'Lise Sehir', 'Lise Turu', 'Lise Bolumu',
       'Lise Bolum Diger', 'Lise Mezuniyet Notu',
       'Baska Bir Kurumdan Burs Aliyor mu?', 'Burs Aldigi Baska Kurum',
       'Baska Kurumdan Aldigi Burs Miktari', 'Anne Egitim Durumu',
       'Anne Calisma Durumu', 'Anne Sektor', 'Baba Egitim Durumu',
       'Baba Calisma Durumu', 'Baba Sektor', 'Kardes Sayisi',
       'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
       'Uye Oldugunuz Kulubun Ismi',
       'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
       'Spor Dalindaki Rolunuz Nedir?', 'Aktif olarak bir STK üyesi misiniz?',
       'Hangi STK'nin Uyesisiniz?', 

In [5]:
drop_columns = [
    'Dogum Tarihi',
    'Dogum Yeri',
    'Ikametgah Sehri',
    'Universite Adi',
    'Bölüm',
    'Lise Adi',
    'Lise Adi Diger',
    'Lise Sehir',
    'Lise Bolumu',
    'Lise Bolum Diger',
    'Burslu ise Burs Yuzdesi',
    'Burs Aldigi Baska Kurum',
    'Baska Kurumdan Aldigi Burs Miktari',
    'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Uye Oldugunuz Kulubun Ismi',
    'Stk Projesine Katildiniz Mi?',
    'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?',
    'Ingilizce Seviyeniz?',
    "Hangi STK'nin Uyesisiniz?",
    'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite',
    'Anne Egitim Durumu',
    'Baba Egitim Durumu'
]
train.drop(columns=drop_columns, inplace=True)

In [6]:
drop_columns = [
    'Dogum Tarihi',
    'Dogum Yeri',
    'Ikametgah Sehri',
    'Universite Adi',
    'Bölüm',
    'Lise Adi',
    'Lise Adi Diger',
    'Lise Sehir',
    'Lise Bolumu',
    'Lise Bolum Diger',
    'Burslu ise Burs Yuzdesi',
    'Burs Aldigi Baska Kurum',
    'Baska Kurumdan Aldigi Burs Miktari',
    'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Uye Oldugunuz Kulubun Ismi',
    'Stk Projesine Katildiniz Mi?',
    'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?',
    'Ingilizce Seviyeniz?',
    "Hangi STK'nin Uyesisiniz?",
    'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite',
    'Anne Egitim Durumu',
    'Baba Egitim Durumu'
]
test.drop(columns=drop_columns, inplace=True)

In [7]:
train.Cinsiyet.unique()

array(['Erkek', 'Kadın', 'ERKEK', 'Belirtmek istemiyorum', nan],
      dtype=object)

In [8]:
# Look at the number of unique values in each column
train.nunique()

Basvuru Yili                                                9
Degerlendirme Puani                                        97
Cinsiyet                                                    4
Universite Turu                                             4
Burs Aliyor mu?                                             5
Universite Kacinci Sinif                                   12
Universite Not Ortalamasi                                  20
Lise Turu                                                  16
Lise Mezuniyet Notu                                        19
Baska Bir Kurumdan Burs Aliyor mu?                          2
Anne Calisma Durumu                                         3
Anne Sektor                                                 8
Baba Calisma Durumu                                         3
Baba Sektor                                                 8
Kardes Sayisi                                              28
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?        2
Profesyo

In [9]:
train.isna().sum()

Basvuru Yili                                                0
Degerlendirme Puani                                         1
Cinsiyet                                                  169
Universite Turu                                           255
Burs Aliyor mu?                                             0
Universite Kacinci Sinif                                  374
Universite Not Ortalamasi                                2753
Lise Turu                                                 847
Lise Mezuniyet Notu                                      5917
Baska Bir Kurumdan Burs Aliyor mu?                        231
Anne Calisma Durumu                                     11896
Anne Sektor                                             34537
Baba Calisma Durumu                                     12618
Baba Sektor                                             20378
Kardes Sayisi                                            1132
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?     1829
Profesyo

In [10]:
# Eksik değerler olan kategorik sütunlar için mod ile doldur
train['Lise Mezuniyet Notu'].fillna(train['Lise Mezuniyet Notu'].mode()[0], inplace=True)
train['Universite Not Ortalamasi'].fillna(train['Universite Not Ortalamasi'].mode()[0], inplace=True)
train['Degerlendirme Puani'].fillna(train['Degerlendirme Puani'].mode()[0], inplace=True)
train['Cinsiyet'].fillna(train['Cinsiyet'].mode()[0], inplace=True)
train['Universite Turu'].fillna(train['Universite Turu'].mode()[0], inplace=True)
train['Universite Kacinci Sinif'].fillna(train['Universite Kacinci Sinif'].mode()[0], inplace=True)
train['Lise Turu'].fillna(train['Lise Turu'].mode()[0], inplace=True)
train['Baska Bir Kurumdan Burs Aliyor mu?'].fillna(train['Baska Bir Kurumdan Burs Aliyor mu?'].mode()[0], inplace=True)
train['Ingilizce Biliyor musunuz?'].fillna(train['Ingilizce Biliyor musunuz?'].mode()[0], inplace=True)

In [11]:
train.isna().sum()

Basvuru Yili                                                0
Degerlendirme Puani                                         0
Cinsiyet                                                    0
Universite Turu                                             0
Burs Aliyor mu?                                             0
Universite Kacinci Sinif                                    0
Universite Not Ortalamasi                                   0
Lise Turu                                                   0
Lise Mezuniyet Notu                                         0
Baska Bir Kurumdan Burs Aliyor mu?                          0
Anne Calisma Durumu                                     11896
Anne Sektor                                             34537
Baba Calisma Durumu                                     12618
Baba Sektor                                             20378
Kardes Sayisi                                            1132
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?     1829
Profesyo

In [12]:
test.isna().sum()

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [13]:
import pandas as pd

# Sayısal sütunlar için örnek değerler ile doldurma
# Eksik veriler için değerler genel bir yaklaşım olabilir
# Verilerinizin türüne göre değişiklik gösterebilir
numerical_columns = {
    'Burslu ise Burs Yuzdesi': -1,  # Eksik burs yüzdesi için örnek bir değer
    'Daha Once Baska Bir Universiteden Mezun Olmus': 'bilinmiyor',  # Eksik değerler için
    'Lise Adi Diger': 'bilinmiyor',
    'Lise Bolum Diger': 'bilinmiyor',
    'Ingilizce Seviyeniz?': 'bilinmiyor',
    'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite': 'bilinmiyor'
}

# Kategorik sütunlar için mod ile doldurma
categorical_columns = [
    'Burslu ise Burs Yuzdesi',
    'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Lise Adi Diger',
    'Lise Bolum Diger',
    'Ingilizce Seviyeniz?',
    'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'
]

# Sayısal sütunlar için medyan veya özel değerler ile doldurma
for col, value in numerical_columns.items():
    if col in test.columns:
        test[col] = pd.to_numeric(test[col], errors='coerce')  # Sayısal değerlere dönüştür
        test[col].fillna(value, inplace=True)

# Kategorik sütunlar için mod ile doldurma
for col in categorical_columns:
    if col in test.columns:
        mode_value = test[col].mode()
        if not mode_value.empty:
            test[col].fillna(mode_value[0], inplace=True)
        else:
            print(f"{col} sütununda mod bulunamadı.")

# Eksik değerlerin olup olmadığını kontrol edin
print(test.isna().sum())

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 22 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          65125 non-null  int64  
 1   Degerlendirme Puani                                   65125 non-null  float64
 2   Cinsiyet                                              65125 non-null  object 
 3   Universite Turu                                       65125 non-null  object 
 4   Burs Aliyor mu?                                       65125 non-null  object 
 5   Universite Kacinci Sinif                              65125 non-null  object 
 6   Universite Not Ortalamasi                             65125 non-null  object 
 7   Lise Turu                                             65125 non-null  object 
 8   Lise Mezuniyet Notu                                   65

In [15]:
from sklearn.preprocessing import LabelEncoder

# Nesne veri türüne sahip sütunları seçme
object_columns = ['Cinsiyet', 'Universite Turu', 'Burs Aliyor mu?', 'Universite Kacinci Sinif',                               
'Universite Not Ortalamasi', 'Lise Turu']

# LabelEncoder nesnelerini depolamak için bir sözlük oluşturma
le_dict = {}

# Her bir nesne sütunu için LabelEncoder uygulama
for column in object_columns:
    le = LabelEncoder()
    train[column] = le.fit_transform(train[column])
    le_dict[column] = le

# Dönüştürülmüş DataFrame'i gösterme
train.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Universite Turu,Burs Aliyor mu?,Universite Kacinci Sinif,Universite Not Ortalamasi,Lise Turu,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,...,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?,id
0,2014,52.0,2,3,1,3,13,1,3.50-3,Hayır,...,Emekli,0,1,Evet,Evet,Diğer,Hayır,Hayır,Evet,0
1,2014,30.0,2,3,2,3,11,1,3.00 - 4.00,Hayır,...,Emekli,0,1,Hayır,Hayır,0,Hayır,Evet,Evet,1
2,2014,18.0,2,3,2,1,11,10,3.50-3,Hayır,...,Evet,Özel Sektör,6,Evet,Hayır,0,Hayır,Hayır,Evet,2
3,2014,40.0,2,3,1,3,11,7,3.50-3,Hayır,...,Emekli,0,10,Evet,Hayır,0,Hayır,Hayır,Evet,3
4,2014,24.0,2,3,1,2,11,14,3.00 - 4.00,Hayır,...,Emekli,0,1,Hayır,Hayır,0,Hayır,Hayır,Evet,4


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 22 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          65125 non-null  int64  
 1   Degerlendirme Puani                                   65125 non-null  float64
 2   Cinsiyet                                              65125 non-null  int32  
 3   Universite Turu                                       65125 non-null  int32  
 4   Burs Aliyor mu?                                       65125 non-null  int32  
 5   Universite Kacinci Sinif                              65125 non-null  int32  
 6   Universite Not Ortalamasi                             65125 non-null  int32  
 7   Lise Turu                                             65125 non-null  int32  
 8   Lise Mezuniyet Notu                                   65

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 21 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          11049 non-null  int64  
 1   Cinsiyet                                              11049 non-null  object 
 2   Universite Turu                                       11049 non-null  object 
 3   Burs Aliyor mu?                                       11049 non-null  object 
 4   Universite Kacinci Sinif                              11049 non-null  object 
 5   Universite Not Ortalamasi                             11049 non-null  object 
 6   Lise Turu                                             11049 non-null  object 
 7   Lise Mezuniyet Notu                                   11049 non-null  object 
 8   Baska Bir Kurumdan Burs Aliyor mu?                    11

In [18]:
from sklearn.preprocessing import LabelEncoder

# Nesne veri türüne sahip sütunları seçme
object_columns = ['Cinsiyet', 'Universite Turu', 'Burs Aliyor mu?', 'Universite Kacinci Sinif',                               
'Universite Not Ortalamasi', 'Lise Turu']

# LabelEncoder nesnelerini depolamak için bir sözlük oluşturma
le_dict = {}

# Her bir nesne sütunu için LabelEncoder uygulama
for column in object_columns:
    le = LabelEncoder()
    test[column] = le.fit_transform(test[column])
    le_dict[column] = le

# Dönüştürülmüş DataFrame'i gösterme
test.head()

,Basvuru Yili,Cinsiyet,Universite Turu,Burs Aliyor mu?,Universite Kacinci Sinif,Universite Not Ortalamasi,Lise Turu,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Anne Calisma Durumu,...,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?,id
0,2023,2,0,1,0,2,0,50 - 74,Hayır,Evet,...,Evet,Diğer,2.0,Hayır,Evet,Takım Oyuncusu,Evet,Hayır,Evet,0
1,2023,2,0,1,0,5,0,75 - 100,Hayır,Hayır,...,Evet,Özel Sektör,3.0,Hayır,Evet,Bireysel Spor,Hayır,Hayır,Evet,1
2,2023,1,1,0,0,5,1,50 - 74,Evet,Hayır,...,Hayır,-,0.0,Hayır,Hayır,Bireysel Spor,Hayır,Hayır,Evet,2
3,2023,1,0,1,0,5,0,75 - 100,Hayır,Hayır,...,Evet,Özel Sektör,4.0,Hayır,Hayır,-,Evet,Hayır,Hayır,3
4,2023,1,0,1,2,3,1,75 - 100,Evet,Evet,...,Evet,Kamu,1.0,Hayır,Hayır,-,Evet,Evet,Evet,4


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 21 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          11049 non-null  int64  
 1   Cinsiyet                                              11049 non-null  int32  
 2   Universite Turu                                       11049 non-null  int32  
 3   Burs Aliyor mu?                                       11049 non-null  int32  
 4   Universite Kacinci Sinif                              11049 non-null  int32  
 5   Universite Not Ortalamasi                             11049 non-null  int32  
 6   Lise Turu                                             11049 non-null  int32  
 7   Lise Mezuniyet Notu                                   11049 non-null  object 
 8   Baska Bir Kurumdan Burs Aliyor mu?                    11

In [20]:
# Create input properties by dropping id and class columns
X = train.drop(['id', 'Degerlendirme Puani'], axis=1)
y = train['Degerlendirme Puani']

# Separating X and y data sets into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [21]:
print(X_train.dtypes)

Basvuru Yili                                             int64
Cinsiyet                                                 int32
Universite Turu                                          int32
Burs Aliyor mu?                                          int32
Universite Kacinci Sinif                                 int32
Universite Not Ortalamasi                                int32
Lise Turu                                                int32
Lise Mezuniyet Notu                                     object
Baska Bir Kurumdan Burs Aliyor mu?                      object
Anne Calisma Durumu                                     object
Anne Sektor                                             object
Baba Calisma Durumu                                     object
Baba Sektor                                             object
Kardes Sayisi                                           object
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    object
Profesyonel Bir Spor Daliyla Mesgul musunuz?           

In [22]:
# Boş stringleri NaN ile değiştirme
X_train.replace('', float('nan'), inplace=True)

# NaN değerleri uygun şekilde doldurma (örneğin, ortalama, medyan, veya belirli bir değerle)
X_train.fillna(X_train.mean(numeric_only=True), inplace=True)

In [23]:
# Kategorik sütunlarda eksik verileri mod ile doldurma
for column in X_train.select_dtypes(include=['object']).columns:
    mode_value = X_train[column].mode()[0]
    X_train[column].fillna(mode_value, inplace=True)

In [24]:
print(X_train.isnull().sum())

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [25]:
from sklearn.preprocessing import LabelEncoder

# Yalnızca object türündeki sütunları dönüştürme
for column in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train[column] = le.fit_transform(X_train[column].astype(str))

In [26]:
# NaN değerlerini kontrol etme
print(X_train.isnull().sum())

# NaN değerleri uygun şekilde doldurma (örneğin, boş string ile)
X_train.fillna('', inplace=True)

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [27]:
from sklearn.preprocessing import LabelEncoder

# Sadece string sütunları dönüştürme
for column in X_train.select_dtypes(include=['object']).columns:
    # Sütunda sadece string veri varsa dönüştür
    if X_train[column].apply(lambda x: isinstance(x, str)).all():
        le = LabelEncoder()
        X_train[column] = le.fit_transform(X_train[column])

In [28]:
print(X_test.dtypes)

Basvuru Yili                                             int64
Cinsiyet                                                 int32
Universite Turu                                          int32
Burs Aliyor mu?                                          int32
Universite Kacinci Sinif                                 int32
Universite Not Ortalamasi                                int32
Lise Turu                                                int32
Lise Mezuniyet Notu                                     object
Baska Bir Kurumdan Burs Aliyor mu?                      object
Anne Calisma Durumu                                     object
Anne Sektor                                             object
Baba Calisma Durumu                                     object
Baba Sektor                                             object
Kardes Sayisi                                           object
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    object
Profesyonel Bir Spor Daliyla Mesgul musunuz?           

In [29]:
# Boş stringleri NaN ile değiştirme
X_test.replace('', float('nan'), inplace=True)

# NaN değerleri uygun şekilde doldurma (örneğin, ortalama, medyan, veya belirli bir değerle)
X_test.fillna(X_test.mean(numeric_only=True), inplace=True)

In [30]:
# Kategorik sütunlarda eksik verileri mod ile doldurma
for column in X_test.select_dtypes(include=['object']).columns:
    mode_value = X_test[column].mode()[0]
    X_test[column].fillna(mode_value, inplace=True)

In [31]:
print(X_test.isnull().sum())

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [32]:
from sklearn.preprocessing import LabelEncoder

# Yalnızca object türündeki sütunları dönüştürme
for column in X_test.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_test[column] = le.fit_transform(X_test[column].astype(str))

In [33]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 5, max_samples = 0.2, random_state = 0)

regressor.fit(X_train, y_train)

RandomForestRegressor(max_samples=0.2, n_estimators=5, random_state=0)

In [34]:
# Making predictions based on input features (X_test) in the test set using a trained classifier model
y_pred = regressor.predict(X_test)
y_pred

array([30.2, 16.4, 45.4, ..., 37. , 43. , 55.2])

In [35]:
# Remove 'id' column from DataFrame named test
test = test.drop(['id'], axis=1)

In [36]:
print(test.dtypes)

Basvuru Yili                                              int64
Cinsiyet                                                  int32
Universite Turu                                           int32
Burs Aliyor mu?                                           int32
Universite Kacinci Sinif                                  int32
Universite Not Ortalamasi                                 int32
Lise Turu                                                 int32
Lise Mezuniyet Notu                                      object
Baska Bir Kurumdan Burs Aliyor mu?                       object
Anne Calisma Durumu                                      object
Anne Sektor                                              object
Baba Calisma Durumu                                      object
Baba Sektor                                              object
Kardes Sayisi                                           float64
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?     object
Profesyonel Bir Spor Daliyla Mesgul musu

In [37]:
# Boş stringleri NaN ile değiştirme
test.replace('', float('nan'), inplace=True)

# NaN değerleri uygun şekilde doldurma (örneğin, ortalama, medyan, veya belirli bir değerle)
test.fillna(test.mean(numeric_only=True), inplace=True)

In [38]:
print(test.isnull().sum())

Basvuru Yili                                            0
Cinsiyet                                                0
Universite Turu                                         0
Burs Aliyor mu?                                         0
Universite Kacinci Sinif                                0
Universite Not Ortalamasi                               0
Lise Turu                                               0
Lise Mezuniyet Notu                                     0
Baska Bir Kurumdan Burs Aliyor mu?                      0
Anne Calisma Durumu                                     0
Anne Sektor                                             0
Baba Calisma Durumu                                     0
Baba Sektor                                             0
Kardes Sayisi                                           0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?    0
Profesyonel Bir Spor Daliyla Mesgul musunuz?            0
Spor Dalindaki Rolunuz Nedir?                           0
Aktif olarak b

In [39]:
from sklearn.preprocessing import LabelEncoder

# Yalnızca object türündeki sütunları dönüştürme
for column in test.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    test[column] = le.fit_transform(test[column].astype(str))

In [40]:
# Making predictions based on input features (test) in the test set using a trained regressor model
y_predX = regressor.predict(test)
y_predX

array([33.6, 11.2, 22.2, ..., 46.8, 41.8, 24. ])

In [41]:
# Importing the data
submission = pd.read_csv('sample_submission.csv')
submission

,id,Degerlendirme Puani
0,0,1.0
1,1,27.0
2,2,17.0


In [42]:
test2 = pd.read_csv('test.csv')
test.head()

,Basvuru Yili,Cinsiyet,Universite Turu,Burs Aliyor mu?,Universite Kacinci Sinif,Universite Not Ortalamasi,Lise Turu,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Anne Calisma Durumu,Anne Sektor,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?
0,2023,2,0,1,0,2,0,1,1,0,3,0,1,2.0,1,0,4,0,1,0
1,2023,2,0,1,0,5,0,2,1,1,0,0,3,3.0,1,0,1,1,1,0
2,2023,1,1,0,0,5,1,1,0,1,0,1,0,0.0,1,1,1,1,1,0
3,2023,1,0,1,0,5,0,2,1,1,0,0,3,4.0,1,1,0,0,1,1
4,2023,1,0,1,2,3,1,2,0,0,2,0,2,1.0,1,1,0,0,0,0


In [43]:
result_df = pd.DataFrame({
        "id": test2['id'],
        "Degerlendirme Puani": y_predX
    })

In [44]:
result_df

,id,Degerlendirme Puani
0,0,33.6
1,1,11.2
2,2,22.2
3,3,21.0
4,4,31.4
...,...,...
11044,11044,28.4
11045,11045,26.6
11046,11046,46.8
11047,11047,41.8


In [45]:
# result_df.to_csv('submission7.csv', index=False)